In [1]:
import torch
import torchvision
from torch import optim 
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms,datasets

In [2]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(64),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(64),
                                      transforms.CenterCrop(64),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder('/home/nilesh/Desktop/MY FILES/hackathon-dataset/raw/apple_data/training', transform=train_transforms)
test_data = datasets.ImageFolder('/home/nilesh/Desktop/MY FILES/hackathon-dataset/raw/apple_data/testing', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size= 600 ,shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=30 , shuffle = True)

In [3]:
_ , (images , labels) = next(enumerate(trainloader))

In [4]:
images.shape , labels.shape

(torch.Size([600, 3, 64, 64]), torch.Size([600]))

In [5]:
class neural_net(nn.Module):
    def __init__(self):
        super().__init__()
        #64*64*3
        self.conv1 = nn.Conv2d(3 , 32 , 3 , padding = 1)
        #32*32*32
        self.conv2 = nn.Conv2d(32 , 64 , 3 , padding = 1)
        #16*16*64
        self.conv3 = nn.Conv2d(64 , 32 , 3 , padding = 1)
        #8*8*32
        self.dropout = nn.Dropout(0.2)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(8*8*32 , 100)
        self.fc2 = nn.Linear(100 , 64)
        self.fc3 = nn.Linear(64 , 32)
        self.fc4 = nn.Linear(32 , 4)
        
        
    def forward(self , x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1 , 8*8*32)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x) , dim = 1)
        return x

In [6]:
model = neural_net()
model(images).shape

torch.Size([600, 4])

In [7]:
model

neural_net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout): Dropout(p=0.2)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=4, bias=True)
)

In [8]:
labels.shape

torch.Size([600])

In [9]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters() , lr = 0.01)

In [10]:
for epoch in range(300):
    optimizer.zero_grad()
    output = model(images)
    loss = criterion(output , labels)
    loss.backward()
    optimizer.step()
    print('epoch' , epoch+1 , ': ' ,loss.item())

epoch 1 :  1.3903584480285645
epoch 2 :  1.4052120447158813
epoch 3 :  1.4103714227676392
epoch 4 :  1.3729112148284912
epoch 5 :  1.3167728185653687
epoch 6 :  2.406002998352051
epoch 7 :  1.2771538496017456
epoch 8 :  1.2908447980880737
epoch 9 :  1.2527920007705688
epoch 10 :  1.2339307069778442
epoch 11 :  1.2202439308166504
epoch 12 :  1.2000632286071777
epoch 13 :  1.1535521745681763
epoch 14 :  1.0538461208343506
epoch 15 :  1.0827223062515259
epoch 16 :  1.321999430656433
epoch 17 :  1.388856291770935
epoch 18 :  0.9144985675811768
epoch 19 :  1.0560225248336792
epoch 20 :  1.008840560913086
epoch 21 :  1.0070688724517822
epoch 22 :  1.0237065553665161
epoch 23 :  1.038648009300232
epoch 24 :  1.013755202293396
epoch 25 :  1.0001152753829956
epoch 26 :  0.9475778341293335
epoch 27 :  0.9386507272720337
epoch 28 :  0.8644523024559021
epoch 29 :  0.8589496612548828
epoch 30 :  0.7806224822998047
epoch 31 :  0.7501751184463501
epoch 32 :  0.7518090605735779
epoch 33 :  0.767037332

epoch 252 :  0.0009131181286647916
epoch 253 :  0.0028461976908147335
epoch 254 :  0.0013656584778800607
epoch 255 :  0.0018028473714366555
epoch 256 :  0.001033708220347762
epoch 257 :  0.002522779628634453
epoch 258 :  0.0036107730120420456
epoch 259 :  0.01569407433271408
epoch 260 :  0.0005337190814316273
epoch 261 :  0.015370087698101997
epoch 262 :  0.017603019252419472
epoch 263 :  0.006113184150308371
epoch 264 :  0.0011318882461637259
epoch 265 :  0.006473087705671787
epoch 266 :  0.016775496304035187
epoch 267 :  0.000766850309446454
epoch 268 :  0.0005100623820908368
epoch 269 :  0.003970366902649403
epoch 270 :  0.11490260809659958
epoch 271 :  0.0007948168204165995
epoch 272 :  0.01266928855329752
epoch 273 :  0.06929776817560196
epoch 274 :  0.032360587269067764
epoch 275 :  0.014176413416862488
epoch 276 :  0.03510502353310585
epoch 277 :  0.02936345338821411
epoch 278 :  0.013489956967532635
epoch 279 :  0.005708457436412573
epoch 280 :  0.010232516564428806
epoch 281 :

In [11]:
train_predictions = torch.exp(model(images)).detach().numpy()

In [12]:
final_train_predictions = []
for pred in train_predictions:
    final_train_predictions.append(pred.argmax())
final_train_predictions

[1,
 3,
 3,
 1,
 3,
 0,
 3,
 1,
 2,
 0,
 3,
 0,
 3,
 3,
 0,
 3,
 3,
 2,
 1,
 1,
 0,
 0,
 2,
 2,
 0,
 3,
 2,
 3,
 3,
 3,
 3,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 3,
 2,
 2,
 0,
 3,
 2,
 2,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 2,
 2,
 1,
 2,
 0,
 1,
 0,
 2,
 0,
 0,
 2,
 1,
 1,
 2,
 2,
 2,
 3,
 0,
 3,
 1,
 3,
 2,
 2,
 1,
 1,
 2,
 3,
 0,
 2,
 0,
 1,
 1,
 0,
 1,
 3,
 0,
 3,
 0,
 1,
 0,
 1,
 3,
 2,
 1,
 2,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 3,
 2,
 3,
 1,
 3,
 3,
 3,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 3,
 0,
 3,
 3,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 3,
 1,
 0,
 0,
 3,
 1,
 2,
 3,
 0,
 0,
 2,
 1,
 0,
 2,
 2,
 2,
 1,
 1,
 3,
 3,
 3,
 0,
 3,
 2,
 2,
 1,
 2,
 2,
 3,
 2,
 3,
 3,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 3,
 2,
 3,
 0,
 0,
 0,
 1,
 1,
 2,
 2,
 3,
 0,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 3,
 1,
 2,
 2,
 3,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 3,
 3,
 1,
 3,
 1,
 1,
 3,
 3,
 3,
 0,
 2,
 3,
 1,
 0,
 0,
 3,
 3,
 1,
 0,
 1,
 0,
 3,
 3,
 2,
 2,
 2,
 3,
 2,
 2,
 0,
 0,
 3,
 1,
 2,
 1,
 2,
 2,
 3,
 0,
 3,


In [13]:
len(labels)

600

In [14]:
print('train accuracy : ' , (final_train_predictions == labels.detach().numpy()).sum()/len(final_train_predictions))

train accuracy :  1.0


In [15]:
_ , (test_img , test_label) = next(enumerate(testloader))

In [16]:
test_label

tensor([3, 2, 3, 2, 0, 1, 1, 2, 3, 0, 0, 3, 1, 2, 2, 0, 2, 0, 3, 1, 2, 1, 0, 1,
        2, 2, 3, 0, 2, 3])

In [17]:
prediction = torch.exp(model(test_img)).detach().numpy()

In [18]:
final_predictions = []
for pred in prediction:
    final_predictions.append(pred.argmax())
final_predictions

[3,
 2,
 3,
 2,
 0,
 1,
 1,
 2,
 3,
 0,
 0,
 3,
 1,
 2,
 2,
 0,
 2,
 0,
 3,
 1,
 2,
 1,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 3]

In [19]:
print('test predictions : ' , (final_predictions == test_label.detach().numpy()).sum()/len(final_predictions))

test predictions :  0.9333333333333333


In [20]:
import pickle

In [22]:
filename = 'finalized_model1.sav'

In [23]:
pickle.dump(model , open(filename , 'wb'))

In [24]:
loaded_model = pickle.load(open(filename ,'rb'))

In [25]:
loaded_model

neural_net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout): Dropout(p=0.2)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=4, bias=True)
)

In [26]:
pred = torch.exp(loaded_model(test_img)).detach().numpy()

In [27]:
final_predictions = []
for pr in pred:
    final_predictions.append(pr.argmax())
final_predictions

[3,
 2,
 0,
 2,
 0,
 1,
 1,
 2,
 3,
 0,
 0,
 3,
 1,
 2,
 2,
 0,
 2,
 0,
 3,
 1,
 2,
 1,
 0,
 1,
 2,
 2,
 0,
 0,
 2,
 0]

In [28]:
test_label

tensor([3, 2, 3, 2, 0, 1, 1, 2, 3, 0, 0, 3, 1, 2, 2, 0, 2, 0, 3, 1, 2, 1, 0, 1,
        2, 2, 3, 0, 2, 3])

In [29]:
(final_predictions == test_label.detach().numpy()).sum()/len(final_predictions)

0.9

In [36]:
import os

In [37]:
def find_classes(dir):
    classes = os.listdir(dir)
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx

In [38]:
find_classes('/home/nilesh/Desktop/MY FILES/hackathon-dataset/raw/apple_data/training')

(['healthy', 'rot', 'rust', 'scab'],
 {'healthy': 0, 'rot': 1, 'rust': 2, 'scab': 3})